In [1]:
import folium
import osmnx as ox
import io
import zipfile
import requests
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings

# Import the data
url = "https://api.data.gov.my/gtfs-static/prasarana?category=rapid-rail-kl"
resp = requests.get(url, stream=True)
resp.raise_for_status()
zfile = zipfile.ZipFile(io.BytesIO(resp.content))

mydf = pd.read_csv(zfile.open(f'{zfile.namelist()[-2]}'))

mydf['Station'] = mydf['stop_id'] + ' ' + mydf['stop_name']
station = sorted(mydf['Station'])

# Set the walking speed
sp_walk = 1.5  # in m/s
# Set the cycling speed
sp_cycling = 5  # in m/s
# Set the driving speed
sp_drive = 10  # in m/s

# Dropdown for Station
dropdown_station = widgets.Dropdown(
    options=station,
    description='Station:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='35%', align_self='flex-end')
)

# Dropdown for Network Type
dropdown_network = widgets.Dropdown(
    options=["all_private", "all", "bike", "drive", "drive_service", "walk"],
    value = 'walk',
    description='Network Type:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='35%', align_self='flex-end')
)
#"all_private": This includes all streets, including private ones
#'all': This includes all streets that are not explicitly marked as private
#"bike": This includes bike paths and streets suitable for cycling
#"drive": This includes all drivable roads, which typically means streets for cars, trucks, and other motor vehicles
#"drive_service": This includes service roads, typically used for local access to facilities rather than through traffic 
#"walk": This includes all streets and paths suitable for walking

# Widget for Travel time
widge_time = widgets.BoundedFloatText(
    value=5,
    min = 1,
    step= 1,
    description='Travel time (any positive integer, in min):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='35%', align_self='flex-end'),
    disabled=False
)

# Create a button widget
run_button = widgets.Button(description="Run Code")

# Display the widgets
display(dropdown_station, dropdown_network, widge_time, run_button)

# Create an HTML container for the map
map_output = widgets.Output()

# Create a loading progress widget
loading_widget = widgets.IntProgress(
    value=0,
    min=0,
    max=1,
    step=1,
    description='Loading:',
    bar_style='info',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='30%', align_self='flex-end')
)

# Function to update loading status
def update_loading_status(loading=True):
    if loading:
        loading_widget.value = 0  # Reset progress bar
        display(loading_widget)
    else:
        loading_widget.value = 1  # Complete progress bar
        clear_output(wait=True)
        
# Function to get station coordinates
def get_station_coordinates(selected_station):
    station_info = mydf.loc[mydf['Station'] == selected_station]
    return station_info['stop_lat'].values[0], station_info['stop_lon'].values[0]

# Run button:
def run_code(button):
    with map_output:
        clear_output(wait=True)
        
        # Display "Loading" status
        update_loading_status()
        
        # Obtain the latitude and longitude of the train station
        lat, lon = get_station_coordinates(dropdown_station.value)

        # Set the distance
        if dropdown_network.value in ["all_private", "all", "drive", "drive_service"]:
            my_dist = widge_time.value * 60 * sp_drive
        elif dropdown_network.value == 'walk':
            my_dist = widge_time.value * 60 * sp_walk
        elif dropdown_network.value == 'bike':
            my_dist = widge_time.value * 60 * sp_cycling

        # Download/model a street network for the specific station then visualize it
        G = ox.graph.graph_from_point((lat, lon), dist=my_dist, dist_type='network', network_type=dropdown_network.value)
        ## 'network': actual distance along the street network
        ## 'bbox': radius of a bounding box around the central point
        #fig, ax = ox.plot_graph(G)

        # Create a folium map
        mymap = folium.Map(location=[lat, lon], dragging=True, scrollWheelZoom=True)
        
        # Plot the OpenStreetMap graph on the map
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            ox.plot_graph_folium(G, graph_map=mymap, edge_color='blue', edge_width=2)

            # Add a CircleMarker to the map
            folium.CircleMarker(location=[lat, lon], fill=True, color='orange', radius=9,
                                popup='Station: {}'.format(dropdown_station.value)).add_to(mymap)

            display(mymap)
            
        # Complete "loading" status after the map is displayed
        update_loading_status(False)

# Associate the callback function to the button's on_click event
run_button.on_click(run_code)

# Display the HTML container for the map
display(map_output)


Dropdown(description='Station:', layout=Layout(align_self='flex-end', width='35%'), options=('AG1 SENTUL TIMUR…

Dropdown(description='Network Type:', index=5, layout=Layout(align_self='flex-end', width='35%'), options=('al…

BoundedFloatText(value=5.0, description='Travel time (any positive integer, in min):', layout=Layout(align_sel…

Button(description='Run Code', style=ButtonStyle())

Output()

In [9]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p folium,osmnx,io,zipfile,requests,pandas,ipywidgets,IPython,warnings,watermark

# date
print (" ")
%watermark -u -n -t -z 



The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

folium    : 0.14.0
osmnx     : 1.7.1
io        : unknown
zipfile   : unknown
requests  : 2.31.0
pandas    : 1.1.3
ipywidgets: 7.5.1
IPython   : 7.19.0
warnings  : unknown
watermark : 2.4.3

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
CPU cores   : 4
Architecture: 64bit

 
Last updated: Thu Dec 07 2023 14:01:50Malay Peninsula Standard Time

